In [49]:
# 把之前的一些操作刪掉
# 刪掉所有Namespaces
! kubectl delete namespaces --all
# 刪掉所有資源
! kubectl delete all --all 
# 把預設 namespace 恢復回 default
! kubectl config set-context --current --namespace=default

namespace "kube-node-lease" deleted
Error from server (Forbidden): namespaces "default" is forbidden: this namespace may not be deleted
Error from server (Forbidden): namespaces "kube-public" is forbidden: this namespace may not be deleted
Error from server (Forbidden): namespaces "kube-system" is forbidden: this namespace may not be deleted
pod "hostname-695cf7968c-f9pvp" deleted
service "kubernetes" deleted
deployment.apps "hostname" deleted
replicaset.apps "hostname-695cf7968c" deleted
Context "minikube" modified.


# Namespaces 

- Namespaces 提供一個方式把資源隔離
- kubectl 的默認 namespaces 是 `default`
- 用法：同一個應用使用共同Namespaces


In [30]:
# 查看目前使用的預設Namesapce名稱
! kubectl config get-contexts 

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   development


In [31]:
# 列出目前有的所有Namespaces 
! kubectl get namespaces

NAME              STATUS   AGE
default           Active   41m
kube-node-lease   Active   3m20s
kube-public       Active   41m
kube-system       Active   41m


## initial namespaces

minikube(kubernetes)有4個內建的Namespaces

- `default`: 默認namespaces
- `kube-node-lease`: 處理lease租約。# 空的
- `kube-public`: 公開namespaces, 爲經認證的節點也可以存取。# 空的
- `kube-system`: created by kube-system, 包含維持kubernetes正常運行的deployment

In [42]:
# 列印目前所有資源 (default)
# 沒有資源是正常的，因爲你什麼資源都沒建立
! kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/hostname-695cf7968c-f9pvp   1/1     Running   0          51m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   52m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hostname   1/1     1            1           51m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/hostname-695cf7968c   1         1         1       51m


In [43]:
! kubectl get all -n kube-node-lease

No resources found in kube-node-lease namespace.


In [44]:
! kubectl get all -n kube-public

NAME                            READY   STATUS    RESTARTS   AGE
pod/hostname-695cf7968c-9t4cs   1/1     Running   0          47m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hostname   1/1     1            1           47m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/hostname-695cf7968c   1         1         1       47m


In [45]:
# 列印在 kube-system namespaces 的所有資源
! kubectl get all -n kube-system

NAME                                   READY   STATUS    RESTARTS      AGE
pod/coredns-565d847f94-9n5hd           1/1     Running   0             51m
pod/etcd-minikube                      1/1     Running   0             52m
pod/kube-apiserver-minikube            1/1     Running   0             52m
pod/kube-controller-manager-minikube   1/1     Running   0             52m
pod/kube-proxy-t4pv4                   1/1     Running   0             51m
pod/kube-scheduler-minikube            1/1     Running   0             52m
pod/storage-provisioner                1/1     Running   1 (51m ago)   52m

NAME               TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)                  AGE
service/kube-dns   ClusterIP   10.96.0.10   <none>        53/UDP,53/TCP,9153/TCP   52m

NAME                        DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR            AGE
daemonset.apps/kube-proxy   1         1         1       1            1           kubernetes.io/os=linux   52m

NAM

In [46]:
# 建立一個Deploy # 沒有制度Namespace
! kubectl create deployment hostname \
  --image=docker.io/brokenpen/hostname:v1

error: failed to create deployment: deployments.apps "hostname" already exists


In [47]:
# 把 deployment.apps/hostname 加上 env=prod 這個標籤
! kubectl label \
  --overwrite deployment hostname env=prod

deployment.apps/hostname not labeled


In [48]:
# 列印 deployment.apps/hostname的資訊
! kubectl get deploy hostname \
  -o custom-columns="Kind:.kind,Namespace:.metadata.namespace, Name:.metadata.name, Labels:.metadata.labels"

Kind         Namespace    Name       Labels
Deployment   default     hostname   map[app:hostname env:prod]


In [11]:
# 描述 deployment.apps/hostname
! kubectl describe deployment hostname 

Name:                   hostname
Namespace:              default
CreationTimestamp:      Wed, 30 Nov 2022 06:36:29 +0000
Labels:                 app=hostname
                        env=prod
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=hostname
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=hostname
  Containers:
   hostname:
    Image:        docker.io/brokenpen/hostname:v1
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSets:  <none>
NewReplicaSet:   hostname-695cf7968c (1/1 replicas created)
Even

In [12]:
# 建立資源時指定 namespaces 爲 kube-public
! kubectl create deployment hostname \
  --image=docker.io/brokenpen/hostname:v1 \
  --namespace=kube-public

error: failed to create deployment: deployments.apps "hostname" already exists


In [13]:
# 列印目前所有 hostname deployment
# 包含所有 namespaces deployment
! kubectl get deploy \
  --all-namespaces \
  --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE
default       hostname   1/1     1            1           37m
kube-public   hostname   1/1     1            1           33m


---

---


#### 目前還只是使用 minikube 的預設 namespace

#### 嘗試下自己建立一個新的 namespaces 

In [14]:
# 建立一個 namesapces 爲 developemnt 的 namespaces 
! kubectl create namespace development

Error from server (AlreadyExists): object is being deleted: namespaces "development" already exists


In [15]:
# 幫剛出建立的 namesapces
# 增加一個 labels 
! kubectl label \
  --overwrite namespace development env=dev

namespace/development not labeled


In [16]:
# 取得 namesapce development 的 yaml
! kubectl get namespaces development -o yaml

apiVersion: v1
kind: Namespace
metadata:
  creationTimestamp: "2022-11-30T06:50:12Z"
  deletionTimestamp: "2022-11-30T07:13:19Z"
  labels:
    env: dev
    kubernetes.io/metadata.name: development
  name: development
  resourceVersion: "2015"
  uid: 26657f71-0081-43eb-82d6-c6d605ea9207
spec:
  finalizers:
  - kubernetes
status:
  conditions:
  - lastTransitionTime: "2022-11-30T07:13:25Z"
    message: All resources successfully discovered
    reason: ResourcesDiscovered
    status: "False"
    type: NamespaceDeletionDiscoveryFailure
  - lastTransitionTime: "2022-11-30T07:13:25Z"
    message: All legacy kube types successfully parsed
    reason: ParsedGroupVersions
    status: "False"
    type: NamespaceDeletionGroupVersionParsingFailure
  - lastTransitionTime: "2022-11-30T07:13:25Z"
    message: All content successfully deleted, may be waiting on finalization
    reason: ContentDeleted
    status: "False"
    type: NamespaceDeletionContentFailure
  - lastTransitionTime: "2022-11-30T07:1

---
- namespace.yaml
  - apiVersion: v1
  - kind: Namespaces
  - metadata:
     - name: development
---

In [17]:
# 把 目前 kubectl 的預設namesapce 改爲 development
! kubectl config set-context \
  --current --namespace="development"

Context "minikube" modified.


In [18]:
# 列印目前的 config contexts
# 想要知道目前的Namesapces是否已改爲 development
! kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   development


In [19]:
# 在新的 namespaces 建立 新的 deployment
! kubectl create deployment hostname\
  --image=docker.io/brokenpen/hostname:v1

error: failed to create deployment: deployments.apps "hostname" is forbidden: unable to create new content in namespace development because it is being terminated


In [20]:
# 在新的 namespaces 建立 新的 deployment
! kubectl create deployment hostname\
  --image=docker.io/brokenpen/hostname:v1

error: failed to create deployment: deployments.apps "hostname" is forbidden: unable to create new content in namespace development because it is being terminated


In [21]:
# 列印目前(namesapces)的 deployment
! kubectl get deployment --show-labels

No resources found in development namespace.


In [22]:
# 一樣補一下label (不補也行)...
! kubectl label \
  --overwrite deployment hostname env=dev

Error from server (NotFound): namespaces "development" not found


In [23]:
! kubectl get deployment --all-namespaces --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE
default       hostname   1/1     1            1           37m
kube-public   hostname   1/1     1            1           33m


In [24]:
! kubectl get deployment\
  --all-namespaces\
  --show-labels\
  --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE   LABELS
default       hostname   1/1     1            1           37m   app=hostname,env=prod
kube-public   hostname   1/1     1            1           33m   app=hostname


In [25]:
! kubectl get deployment --namespace=default

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
hostname   1/1     1            1           37m


In [26]:
! kubectl get pods -n hostname

No resources found in hostname namespace.


In [27]:
! kubectl config use-context hostname 

error: no context exists with the name: "hostname"
